In [1]:
import ir_datasets
import tqdm
import pyterrier as pt
print(pt.__version__)

pt.init()
from pathlib import Path
import re
import pandas as pd
from pyterrier.measures import RR, nDCG, MAP

0.13.0


Java started and loaded: pyterrier.java, pyterrier.terrier.java [version=5.11 (build: craig.macdonald 2025-01-13 21:29), helper_version=0.0.8]
C:\Users\waded\AppData\Local\Temp\ipykernel_35708\178958555.py:6: DeprecationWarning: Call to deprecated method pt.init(). Deprecated since version 0.11.0.
java is now started automatically with default settings. To force initialisation early, run:
pt.java.init() # optional, forces java initialisation
  pt.init()


In [2]:
antique = ir_datasets.load("antique/test")
print(antique.docs[0])

GenericDoc(doc_id='2020338_0', text="A small group of politicians believed strongly that the fact that Saddam Hussien remained in power after the first Gulf War was a signal of weakness to the rest of the world, one that invited attacks and terrorism. Shortly after taking power with George Bush in 2000 and after the attack on 9/11, they were able to use the terrorist attacks to justify war with Iraq on this basis and exaggerated threats of the development of weapons of mass destruction. The military strength of the U.S. and the brutality of Saddam's regime led them to imagine that the military and political victory would be relatively easy.")


In [3]:
# OPTIONAL - Index the data

def antique_gen(limit=100000):
    lastdoc = 0
    for elem in antique.docs:
        if lastdoc >= limit:
            break
        yield {
            "docno": elem.doc_id,
            "text": elem.text,
        }
        lastdoc += 1

max(len(text.encode("utf-8")) for _, text in antique.docs)
idx_path = Path.cwd() / "indices" / "antique_test"
indexer = pt.IterDictIndexer(
    str(idx_path),
    meta={
        "docno": 20,
        "text": 4096,
    },
    stemmer="porter",
    stopwords="terrier",
)

index_ref = indexer.index(antique_gen())

23:42:21.429 [main] WARN org.terrier.structures.indexing.Indexer -- Adding an empty document to the index (730691_1) - further warnings are suppressed
23:43:39.557 [main] WARN org.terrier.structures.indexing.Indexer -- Indexed 502 empty documents


In [4]:
# Define index paths
index_dir_antique = Path.cwd() / "indices" / "antique_test"
print(index_dir_antique.exists())  # Should print True
print(list(index_dir_antique.iterdir()))  # List the contents
# Load the indexes
#index_msmarco = pt.IndexFactory.of(str(index_dir_msmarco))
index_antique = pt.IndexFactory.of(str(index_dir_antique))

# Use BM25 as the baseline retriever
#retriever_msmarco = pt.BatchRetrieve(index_msmarco, wmodel="BM25")
retriever_antique = pt.BatchRetrieve(index_antique, wmodel="BM25")

True
[WindowsPath('C:/Users/waded/OneDrive/Bureaublad/Information Retrieval/repo/indices/antique_test/data.direct.bf'), WindowsPath('C:/Users/waded/OneDrive/Bureaublad/Information Retrieval/repo/indices/antique_test/data.document.fsarrayfile'), WindowsPath('C:/Users/waded/OneDrive/Bureaublad/Information Retrieval/repo/indices/antique_test/data.inverted.bf'), WindowsPath('C:/Users/waded/OneDrive/Bureaublad/Information Retrieval/repo/indices/antique_test/data.lexicon.fsomapfile'), WindowsPath('C:/Users/waded/OneDrive/Bureaublad/Information Retrieval/repo/indices/antique_test/data.lexicon.fsomaphash'), WindowsPath('C:/Users/waded/OneDrive/Bureaublad/Information Retrieval/repo/indices/antique_test/data.lexicon.fsomapid'), WindowsPath('C:/Users/waded/OneDrive/Bureaublad/Information Retrieval/repo/indices/antique_test/data.meta-0.fsomapfile'), WindowsPath('C:/Users/waded/OneDrive/Bureaublad/Information Retrieval/repo/indices/antique_test/data.meta.idx'), WindowsPath('C:/Users/waded/OneDrive/

C:\Users\waded\AppData\Local\Temp\ipykernel_35708\3402352017.py:11: DeprecationWarning: Call to deprecated class BatchRetrieve. (use pt.terrier.Retriever() instead) -- Deprecated since version 0.11.0.
  retriever_antique = pt.BatchRetrieve(index_antique, wmodel="BM25")


In [5]:
# Convert qrels to DataFrames
#qrels_msmarco = pd.DataFrame(msmarco.qrels_iter())
qrels_antique = pd.DataFrame(antique.qrels_iter())

# Convert queries to DataFrames
#queries_msmarco = pd.DataFrame(msmarco.queries_iter())
queries_antique = pd.DataFrame(antique.queries_iter())

# Rename columns for PyTerrier compatibility
#qrels_msmarco.rename(columns={"query_id": "qid", "doc_id": "docno", "relevance": "label"}, inplace=True)
qrels_antique.rename(columns={"query_id": "qid", "doc_id": "docno", "relevance": "label"}, inplace=True)

#queries_msmarco.rename(columns={"query_id": "qid", "text": "query"}, inplace=True)
queries_antique.rename(columns={"query_id": "qid", "text": "query"}, inplace=True)

In [6]:
def clean_query(query):
    query = query.encode("ascii", "ignore").decode()
    query = query.replace("'", "").replace('"', "").replace("`", "")
    query = re.sub(r"\s+", " ", query).strip()
    return query

#queries_msmarco["query"] = queries_msmarco["query"].apply(clean_query)
queries_antique["query"] = queries_antique["query"].apply(clean_query)

In [7]:
#Model imports
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Load the T5 model
MODEL_ID = "prhegde/t5-query-reformulation-RL"
tokenizer = T5Tokenizer.from_pretrained(MODEL_ID)
model = T5ForConditionalGeneration.from_pretrained(MODEL_ID)
model.eval()


T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [8]:
# Function to rewrite queries
def rewrite_query(query, nsent=1):
    input_ids = tokenizer(query, return_tensors="pt").input_ids
    with torch.no_grad():
        output = model.generate(input_ids, max_length=35, num_beams=1, do_sample=True, repetition_penalty=1.8)
    return tokenizer.decode(output[0], skip_special_tokens=True)


queries_antique["rewritten_query"] = queries_antique["query"].apply(rewrite_query)


In [9]:
bm25_antique = pt.terrier.Retriever(index_antique, wmodel="BM25")


In [10]:
pt.Experiment(
    [bm25_antique],
    queries_antique.rename(columns={"rewritten_query": "query"}),  # Use rewritten queries
    qrels_antique,
    eval_metrics=[RR @ 10, nDCG @ 20, MAP],
)

JavaException: JVM exception occurred: Failed to process qid 3990512 'how can we get concentration onsomething?' -- Lexical error at line 1, column 42.  Encountered: <EOF> after : "" org.terrier.querying.parser.QueryParserException